# IMDB Scraping Data

**Start date:** 9/3/2023 

**Scope:** This analysis has the main goal of praticing cleaning data and peforming an exploratory analysis on this dataset.

In [1]:
#Importing the data
# !kaggle datasets download -d bharatnatrayn/movies-dataset-for-feature-extracion-prediction

In [2]:
# import zipfile

# with zipfile.ZipFile('movies-dataset-for-feature-extracion-prediction.zip', 'r') as zip_ref:
#     zip_ref.extractall()

In [31]:
import pandas as pd
import regex as re
import numpy as np

In [4]:
movies_raw_dataset = pd.read_csv('movies.csv')

In [5]:
movies_raw_dataset.head()

,MOVIES,YEAR,GENRE,RATING,ONE-LINE,STARS,VOTES,RunTime,Gross
0,Blood Red Sky,(2021),"\nAction, Horror, Thriller",6.1,\nA woman with a mysterious illness is forced ...,\n Director:\nPeter Thorwarth\n| \n Star...,"21,062",121.0,NaN
1,Masters of the Universe: Revelation,(2021– ),"\nAnimation, Action, Adventure",5.0,\nThe war for Eternia begins again in what may...,"\n \n Stars:\nChris Wood, \nSara...","17,870",25.0,NaN
2,The Walking Dead,(2010–2022),"\nDrama, Horror, Thriller",8.2,\nSheriff Deputy Rick Grimes wakes up from a c...,"\n \n Stars:\nAndrew Lincoln, \n...","885,805",44.0,NaN
3,Rick and Morty,(2013– ),"\nAnimation, Adventure, Comedy",9.2,\nAn animated series that follows the exploits...,"\n \n Stars:\nJustin Roiland, \n...","414,849",23.0,NaN
4,Army of Thieves,(2021),"\nAction, Crime, Horror",NaN,"\nA prequel, set before the events of Army of ...",\n Director:\nMatthias Schweighöfer\n| \n ...,NaN,NaN,NaN


On a first glance, there are some issues that will need some work:
- The database does not contain only movies but movies and series
- All text fields contain special characters like '\n'
- RunTime column contains the full length of movies but only the episode length for series
- For beautifying purposes, column names also need to be standardized

Let's start with the columns names and then we have a look at the dataset summary and types

In [6]:
movies_raw_dataset.columns = pd.Series(movies_raw_dataset.columns).apply(lambda x: x[0].upper() + x[1:].lower())

In [7]:
movies_raw_dataset.describe()

,Rating,Runtime
count,8179.000000,7041.000000
mean,6.921176,68.688539
std,1.220232,47.258056
min,1.100000,1.000000
25%,6.200000,36.000000
50%,7.100000,60.000000
75%,7.800000,95.000000
max,9.900000,853.000000


Rating column seems fine, as the ratings on IMDB go from 1 to 10. However, 853 for run time seems a little excessive (more than 14 hours). Given that th percentile 75% is 95 minutes, we are likely looking at an outlier here. Let's check it out.

In [8]:
movies_raw_dataset.nlargest(5, columns='Runtime')

,Movies,Year,Genre,Rating,One-line,Stars,Votes,Runtime,Gross
1902,El tiempo entre costuras,(2013–2014),"\nAdventure, Drama, History",8.3,\nSira Quiroga is a young Spanish dressmaker e...,"\n \n Stars:\nAdriana Ugarte, \n...","3,876",853.0,NaN
1081,Soupçons,(2004–2018),"\nDocumentary, Crime, Drama",7.9,\nThe high-profile murder trial of American no...,"\n \n Stars:\nMichael Peterson, ...","20,200",629.0,NaN
2498,The Innocence Files,(2020),"\nDocumentary, Crime",8.0,\nCases of wrongful conviction that the Innoce...,"\n \n Stars:\nPeter Neufeld, \nB...","2,335",573.0,NaN
201,The Haunting of Hill House,(2018),"\nDrama, Horror, Mystery",8.6,"\nFlashing between past and present, a fractur...","\n \n Stars:\nMichiel Huisman, \...","195,117",572.0,NaN
820,Cosmos: A Spacetime Odyssey,(2014),\nDocumentary,9.3,\nAn exploration of our discovery of the laws ...,\n \n Stars:\nNeil deGrasse Tyso...,"114,386",557.0,NaN


So apparently we have mixed values on the RunTime column, with some series having the duration per episode and other having the total duration of the series. We will have to find a way to differentiate these. 

In [9]:
movies_raw_dataset.dtypes

Movies       object
Year         object
Genre        object
Rating      float64
One-line     object
Stars        object
Votes        object
Runtime     float64
Gross        object
dtype: object

Both votes and Gross columns are objects, while they could supposedly by numerical values. Let's have a deeper look at Gross. Since on the head there are no values for this column, we don't really know what is its formatting and if it needs any additional work.

In [10]:
movies_raw_dataset.loc[~movies_raw_dataset['Gross'].isna(), 'Gross'].head(20)

77      $75.47M
85     $402.45M
95      $89.22M
111    $315.54M
125     $57.01M
128    $260.00M
132    $132.38M
143    $167.77M
144    $404.52M
145     $15.07M
156     $70.10M
159    $210.61M
161    $327.48M
165    $390.53M
171    $303.00M
172     $56.63M
175     $58.06M
181    $353.01M
189     $46.89M
191      $7.00M
Name: Gross, dtype: object

So the Gross column will also need some formatting. 

We can now work on the column types and formatting

In [11]:
movies_raw_dataset.dtypes

Movies       object
Year         object
Genre        object
Rating      float64
One-line     object
Stars        object
Votes        object
Runtime     float64
Gross        object
dtype: object

In [12]:
movies_raw_dataset['Year']

0            (2021)
1          (2021– )
2       (2010–2022)
3          (2013– )
4            (2021)
           ...     
9994       (2021– )
9995       (2021– )
9996       (2022– )
9997       (2021– )
9998       (2021– )
Name: Year, Length: 9999, dtype: object

In [13]:
#Remove parenthesis from the Year Column
movies_raw_dataset['Year'] = movies_raw_dataset['Year'].str.replace(r'[()]','')

C:\Users\Hugo\AppData\Local\Temp\ipykernel_16168\546281310.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_raw_dataset['Year'] = movies_raw_dataset['Year'].str.replace(r'[()]','')


In [14]:
#Remove new line char from Genre
movies_raw_dataset['Genre'] = movies_raw_dataset['Genre'].str.strip('\n')

In [15]:
#Remove new line char from One-Line
movies_raw_dataset['One-line'] = movies_raw_dataset['One-line'].str.strip('\n')

In [16]:
#Remove new line and special characters from Stars
movies_raw_dataset['Stars'] = movies_raw_dataset['Stars'].str.replace(r'[\n|]','')

C:\Users\Hugo\AppData\Local\Temp\ipykernel_16168\894442060.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_raw_dataset['Stars'] = movies_raw_dataset['Stars'].str.replace(r'[\n|]','')


In [17]:
#Cast Votes as float to preserve the NaN. Converting to integer would make us have to fill those records and filling with 0 or a different number might skew future analysis
movies_raw_dataset['Votes'] = movies_raw_dataset['Votes'].str.replace(',','').astype(float)

In [18]:
#Get the Gross values correctly formatted
def number_formatter(x:str):
    '''Removes the letter identifier of the number magnitude and multiplies it by the value represented, returning a numerical variable'''
    if 'M' in x:
        return float(x.strip('M'))*1000000
    elif 'k' in x:
        return float(x.strip('k'))*1000
    else:
        return float(x)

In [19]:
movies_raw_dataset['Gross'] = movies_raw_dataset['Gross'].str.strip('$')
movies_raw_dataset['Gross'] = movies_raw_dataset['Gross'].apply(lambda x: number_formatter(str(x)))

In [20]:
movies_raw_dataset.head()

,Movies,Year,Genre,Rating,One-line,Stars,Votes,Runtime,Gross
0,Blood Red Sky,2021,"Action, Horror, Thriller",6.1,A woman with a mysterious illness is forced in...,Director:Peter Thorwarth Stars:Peri Ba...,21062.0,121.0,NaN
1,Masters of the Universe: Revelation,2021–,"Animation, Action, Adventure",5.0,The war for Eternia begins again in what may b...,"Stars:Chris Wood, Sarah Michel...",17870.0,25.0,NaN
2,The Walking Dead,2010–2022,"Drama, Horror, Thriller",8.2,Sheriff Deputy Rick Grimes wakes up from a com...,"Stars:Andrew Lincoln, Norman R...",885805.0,44.0,NaN
3,Rick and Morty,2013–,"Animation, Adventure, Comedy",9.2,An animated series that follows the exploits o...,"Stars:Justin Roiland, Chris Pa...",414849.0,23.0,NaN
4,Army of Thieves,2021,"Action, Crime, Horror",NaN,"A prequel, set before the events of Army of th...",Director:Matthias Schweighöfer Stars:M...,NaN,NaN,NaN


Looking a lot better!

Let's now check for duplicates on the data.

In [21]:
#First we drop full duplicates
movies_raw_dataset.drop_duplicates(inplace=True, ignore_index=True)

In [22]:
#Checking for partial duplicates
movies_raw_dataset[movies_raw_dataset.duplicated(subset=['Movies','Year','Genre'], keep=False)].sort_values(by='Movies')

,Movies,Year,Genre,Rating,One-line,Stars,Votes,Runtime,Gross
8265,13 Reasons Why,2017–2020,"Drama, Mystery, Thriller",5.9,Clay's mental health continues to decline as t...,Director:Sunu Gonera Stars:Dylan Minne...,1420.0,60.0,NaN
8109,13 Reasons Why,2017–2020,"Drama, Mystery, Thriller",6.6,"The school goes into lockdown and Clay, Tony a...",Director:Brenda Strong Stars:Dylan Min...,1694.0,61.0,NaN
8284,13 Reasons Why,2017–2020,"Drama, Mystery, Thriller",6.8,When the dean begins a new investigation and t...,Director:Tommy Lohmann Stars:Dylan Min...,1640.0,60.0,NaN
8108,13 Reasons Why,2017–2020,"Drama, Mystery, Thriller",6.1,The Jensens make the boys take a drug test. Wh...,Director:Brenda Strong Stars:Dylan Min...,1507.0,57.0,NaN
8264,13 Reasons Why,2017–2020,"Drama, Mystery, Thriller",5.6,As the school gears up for the Love Is Love da...,Director:Michael Sucsy Stars:Dylan Min...,1635.0,59.0,NaN
...,...,...,...,...,...,...,...,...,...
8340,ÜberWeihnachten,2020,"Comedy, Drama, Romance",7.2,The Most Wonderful Christmas of the Year - the...,"Stars:Luke Mockridge, Seyneb S...",59.0,45.0,NaN
8339,ÜberWeihnachten,2020,"Comedy, Drama, Romance",7.5,Sausages and Potato Salad - Basti hooks up wit...,"Stars:Luke Mockridge, Seyneb S...",66.0,50.0,NaN
8338,ÜberWeihnachten,2020,"Comedy, Drama, Romance",7.2,"Home Is Where the Tree Is - Bastian, an aspiri...","Stars:Luke Mockridge, Seyneb S...",65.0,48.0,NaN
6352,Far Cry,NaN,"Animation, Action, Adventure",NaN,Plot under wraps. Adaptation of the Ubisoft game.,,NaN,NaN,NaN


It seems like there are series in which each line is an episode because they either have a different plot or a different director/actors. We can group this entries, using the rating and votes average, the sum of the runtime and concatenating the text fields to process later.

In [23]:
aggregations = {
    'Rating':'mean',
    'One-line':'sum',
    'Stars':'sum',
    'Votes':'mean',
    'Runtime':'mean',
    'Gross':'mean'
}

movies_raw_dataset = movies_raw_dataset.groupby(['Movies','Year','Genre']).agg(aggregations).reset_index()

Let's check again now

In [24]:
movies_raw_dataset[movies_raw_dataset.duplicated(subset=['Movies','Year','Genre'], keep=False)].sort_values(by='Movies')

,Movies,Year,Genre,Rating,One-line,Stars,Votes,Runtime,Gross


Perfect! Our dataset looks much more clean!

Now it's time for testing if the data obeys to some defined rules. We have already checked that the rating is between the expected range, but now we need to check for the same thing in year, votes count and range.

To check if the year is in between boundries, let's create two new columns:
- **Starting year** - The year the movie was released or series started
- **Ending year** - The year series ended. In case of movies and series which were released during only one year, ending year shall be the same as starting year 

In [29]:
movies_raw_dataset['Startyear'] = movies_raw_dataset['Year'].apply(lambda x: x.split('–')[0])
movies_raw_dataset['Startyear'] = movies_raw_dataset['Startyear'].astype(int)

ValueError: invalid literal for int() with base 10: 'I'

Apprently some rows contain the value 'I' for the _Year_ column. Let's take a look at those rows.

In [32]:
movies_raw_dataset.loc[movies_raw_dataset['Year']=='I', :]

,Movies,Year,Genre,Rating,One-line,Stars,Votes,Runtime,Gross,Startyear
221,Painkiller,I,"Crime, Drama",NaN,Add a PlotAdd a Plot,"Director:Peter Berg Stars:Uzo Aduba, M...",NaN,NaN,NaN,I
1179,Bubbles,I,"Animation, Comedy",NaN,A chimpanzee tells his story of living with th...,Director:Mark Gustafson Star:Piyali Mu...,NaN,NaN,NaN,I
1601,Dead of Winter,I,Drama,NaN,An Asian American family's weekend in the moun...,Director:Mike Gan,NaN,NaN,NaN,I
1903,Empress,I,"Action, Adventure, Drama",NaN,The wife of a galactic dictator makes the deci...,,NaN,NaN,NaN,I
2813,Janet,I,"Biography, Drama",NaN,A young black woman at the Washington Post and...,Director:Janet Mock,NaN,NaN,NaN,I
2938,Juneteenth,I,Musical,NaN,"A musical about Juneteenth, the",,NaN,NaN,NaN,I
3362,Lou,I,"Crime, Drama, Mystery",NaN,A tough elderly woman living on a remote islan...,Director:Anna Foerster Stars:Jurnee Sm...,NaN,NaN,NaN,I
4087,Paranormal,I,"Drama, Horror, Mystery",NaN,"Depicts the adventures of Dr. Refaat Ismail, a...",,NaN,NaN,NaN,I
5010,Sword Art Online,I,"Action, Adventure, Drama",NaN,The series will follow a brilliant young beta ...,,NaN,NaN,NaN,I
5075,Tell No One,I,Thriller,NaN,A widowed husband learns that his wife may sti...,Director:Guillaume Canet,NaN,NaN,NaN,I


These movies do not have much info, so it seems like the I stands for Incoming. Let's see which rows contain anything but numbers and hyphen

In [41]:
movies_raw_dataset.loc[(movies_raw_dataset['Year'].str.contains('[a-zA-Z]')), :]

,Movies,Year,Genre,Rating,One-line,Stars,Votes,Runtime,Gross,Startyear
413,13th: A Conversation with Oprah Winfrey & Ava ...,2017 Video,"Documentary, Short",7.1,Oprah Winfrey sits down with other to discuss ...,"Stars:Oprah Winfrey, Ava DuVernay",193.0,37.0,NaN,2017 Video
427,2020 Golden Globe Awards,2020 TV Special,Comedy,7.2,Comedian Ricky Gervais brutally roasts Hollywo...,Director:Louis J. Horvitz Stars:Ricky ...,703.0,134.0,NaN,2020 TV Special
433,22nd Annual Mark Twain Prize for American Humo...,2020 TV Special,Comedy,7.5,An all-star lineup of entertainers salute and ...,Director:Chris Robinson Stars:Dave Cha...,1715.0,90.0,NaN,2020 TV Special
465,71st Golden Globe Awards,2014 TV Special,News,7.2,The Hollywood Foreign Press Association's 71 a...,Director:Louis J. Horvitz Stars:Tina F...,888.0,NaN,NaN,2014 TV Special
471,8:46,2020 TV Special,"Short, Comedy",8.1,Dave Chappelle addresses George Floyd's death ...,"Directors:Steven Bognar, Julia Reichert ...",2042.0,27.0,NaN,2020 TV Special
...,...,...,...,...,...,...,...,...,...,...
6314,Zapped,2014 TV Movie,"Comedy, Family, Fantasy",5.2,"When Zoey's mom remarries, Zoey finds it hard ...","Director:Peter DeLuise Stars:Zendaya, ...",4517.0,102.0,NaN,2014 TV Movie
6318,Zeitgeist,2007 Video,"Documentary, History",8.1,"Mythology and belief in society today, present...",Director:Peter Joseph Stars:Chogyam Tr...,48656.0,118.0,NaN,2007 Video
6319,Zeitgeist: Addendum,2008 Video,"Documentary, History, War",8.2,Zeitgeist: Addendum attempts to locate the roo...,Director:Peter Joseph Stars:Jacque Fre...,18071.0,123.0,NaN,2008 Video
6342,Zoolander: Super Model,2016 TV Movie,"Animation, Comedy, Sci-Fi",4.9,"In this animated serialized movie, ridiculousl...",Director:Aaron Augenblick Stars:Ben St...,139.0,84.0,NaN,2016 TV Movie


A lot of entries with more information than the year. Let's look at the unique values on these rows.

In [39]:
movies_raw_dataset.loc[(movies_raw_dataset['Year'].str.contains('[a-zA-Z]')), 'Year'].unique()

array(['2017 Video', '2020 TV Special', '2014 TV Special', '2019 Video',
       '2013 Video', '2008 Video', '2019 TV Movie', '2017 TV Movie',
       '2014 Video', '2015 TV Special', '2016 TV Movie',
       '2019 TV Special', '2018 TV Special', '2017 TV Special',
       '2020 TV Movie', '2021 TV Special', '2016 TV Special',
       '2006 Video', '2014 TV Movie', '2006 TV Movie', 'TV Special',
       '2012 Video', '2021 TV Movie', '2015 Video', '2011 Video',
       '2013 TV Special', '2011 TV Movie', '2021 Video',
       '2012 TV Special', '1992 Video', '1993 TV Special', '1989 Video',
       '2013 TV Movie', '2016 Video', '2011 TV Special', 'TV Movie',
       '2018 TV Movie', '1966 TV Special', '2021 TV Short', 'V', 'Video',
       '2004 Video', '2007 TV Special', '2015 TV Movie', '2020 Video',
       '2008 TV Special', '1995 Video', '1998 Video', '2003 Video',
       '2010 Video', '2005 TV Special', '2001 TV Special',
       '2006 TV Special', '2003 TV Special', '2010 TV Special',
     

Well, at least most of them have the year info, so we will be able to extract it somehow. For now, let's take a look on the ones who do not and see if they are interesting to keep.

In [51]:
no_year_info = movies_raw_dataset.loc[~(movies_raw_dataset['Year'].str.contains('[0-9]')), :]

In [55]:
no_year_info.describe()

,Rating,Votes,Runtime,Gross
count,0.0,0.0,2.000000,0.0
mean,NaN,NaN,94.000000,NaN
std,NaN,NaN,48.083261,NaN
min,NaN,NaN,60.000000,NaN
25%,NaN,NaN,77.000000,NaN
50%,NaN,NaN,94.000000,NaN
75%,NaN,NaN,111.000000,NaN
max,NaN,NaN,128.000000,NaN


Do not seem to be very relevant, as there's barely any information on _Rating_ and _Votes_. Let's drop this from the dataset.

In [53]:
movies_raw_dataset = movies_raw_dataset.loc[(movies_raw_dataset['Year'].str.contains('[0-9]')), :]

From here we can filter only the digits from the _year_ column

In [58]:
movies_raw_dataset['Year'] = movies_raw_dataset['Year'].str.replace('[^0-9\–]','')

C:\Users\Hugo\AppData\Local\Temp\ipykernel_16168\1470456750.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_raw_dataset['Year'] = movies_raw_dataset['Year'].str.replace('[^0-9\–]','')


And now we can proceed as intended

In [65]:
movies_raw_dataset['Year'] = movies_raw_dataset['Year'].str.strip('–')

In [60]:
movies_raw_dataset['Startyear'] = movies_raw_dataset['Year'].apply(lambda x: x.split('–')[0])
movies_raw_dataset['Startyear'] = movies_raw_dataset['Startyear'].astype(int)

In [66]:
movies_raw_dataset['Endyear'] = movies_raw_dataset['Year'].apply(lambda x: x.split('–')[-1])
movies_raw_dataset['Endyear'] = movies_raw_dataset['Endyear'].astype(int)

Check again on our numeric variables

In [68]:
movies_raw_dataset.describe()

,Rating,Votes,Runtime,Gross,Startyear,Endyear
count,5580.000000,5.580000e+03,5067.000000,4.160000e+02,6020.000000,6020.000000
mean,6.663469,2.029844e+04,77.882512,4.307601e+07,2015.530233,2015.882392
std,1.214557,8.192992e+04,50.193815,7.966428e+07,8.293858,8.162056
min,1.100000,5.000000e+00,1.000000,0.000000e+00,1932.000000,1932.000000
25%,5.900000,3.350000e+02,45.000000,1.575000e+05,2015.000000,2015.000000
50%,6.800000,1.439500e+03,80.000000,6.420000e+06,2018.000000,2018.000000
75%,7.500000,6.618250e+03,100.000000,4.689500e+07,2020.000000,2020.000000
max,9.400000,1.713028e+06,853.000000,4.863000e+08,2023.000000,2023.000000


Seems a bit odd that we have a runtime of 1 minutes. Let's take a look at the rows with the smaller runtimes.

In [69]:
movies_raw_dataset.nsmallest(10, 'Runtime')

,Movies,Year,Genre,Rating,One-line,Stars,Votes,Runtime,Gross,Startyear,Endyear
353,Trollhunters,2016–2018,"Animation, Action, Adventure",8.2,"Trolls, goblins, humans, Gumm-Gumms -- all man...",,16.5,1.0,NaN,2016,2018
411,13 Reasons Why: Season 2 Date Announcement Com...,2018,"Short, Drama",7.2,Add a Plot,"Director:Greg Jardin Stars:Alisha Boe,...",197.0,1.0,NaN,2018,2018
658,All Hail King Julien: Happy Birthday to You!,2017,"Animation, Short",4.4,Nobody parties like King Julien. And today he'...,,7.0,1.0,NaN,2017,2017
906,Barbie: Happy Birthday to You!,2017,"Animation, Short",4.1,Get ready to make a wish and have the most ama...,"Stars:Ritesh Rajan, America Young",8.0,1.0,NaN,2017,2017
930,Beat Bugs: Happy Birthday to You!,2017,"Animation, Short",NaN,They say it's your birthday - and there's no o...,,NaN,1.0,NaN,2017,2017
1695,Dinotrux: Happy Birthday to You!,2017,"Animation, Short",NaN,Ty and the rest of the Dinotrux just found out...,,NaN,1.0,NaN,2017,2017
2588,Horsin' Around,2014,"Short, Comedy, Music",7.4,The opening title sequence to the spoof 90s si...,,627.0,1.0,NaN,2014,2014
3118,LEGO Friends: Happy Birthday to You!,2017,"Animation, Short",3.6,Surprise. The gang's all here to celebrate you...,,5.0,1.0,NaN,2017,2017
3119,LEGO Ninjago: Masters of Spinjitzu: Happy Birt...,2017,"Animation, Short, Action",5.9,"LEGO Ninjago heroes Kai, Zane, Jay and Cole wa...",,10.0,1.0,NaN,2017,2017
3627,Miraculous: Ladybug & Cat Noir: Happy Birthday...,2017,"Animation, Short",7.3,"Break dancing, dragons, balloons and a DJ: Lad...",,34.0,1.0,NaN,2017,2017


Most of these are classified as shorts, so it is actually possible that the value is correct.

There are some additional ideas we could implement, but they have significant drawbacks.
- We can try to divided the dataset into a movies dataset and a series dataset. However, our only criteria would have to be the presence of multiple values in the _Year_ column, which would mean that the series went on for several years. This is true for most series, but for series broadcasted within the span of a single year, that classification would be erroneous.
- The _Genre_ column could also be divided, but that could imply a hierarchy and the categories seem o be ordered alphabetically and not by order of importance. As an example, a linear regression model could be influenced by this difference, adding different importance to a genre depending on which column it is.
- Creating columns for the director and stars can be a possibilitiy, but again, we have no information on the order of display. Since it is very unlikely that two different movies/series would have the same cast, having the whole cast could be useless. Isolating the director works in most cases, but won't work on the series episodes that had different directors as we have seen above.

In [70]:
movies_raw_dataset.head()

,Movies,Year,Genre,Rating,One-line,Stars,Votes,Runtime,Gross,Startyear,Endyear
0,13 Reasons Why,2017–2020,"Drama, Mystery, Thriller",6.140000,"The police question Tyler about the guns, leav...",Director:Russell Mulcahy Stars:Dylan M...,1693.600000,62.800000,NaN,2017,2020
1,1899,2022,"Drama, History, Horror",NaN,Add a PlotAdd a Plot,Director:Baran bo Odar Stars:Aneurin B...,NaN,NaN,NaN,2022,2022
2,3Below: Tales of Arcadia,2018–2019,"Animation, Action, Adventure",7.950000,"Left vulnerable after Omen's attack, the royal...",Director:Andrew L. Schmidt Stars:Tatia...,129.500000,20.461538,NaN,2018,2019
3,50M2,2021,"Comedy, Drama, Thriller",7.300000,"While seeking answers about his parents, Shado...",Director:Selçuk Aydemir Stars:Engin Öz...,119.500000,49.000000,NaN,2021,2021
4,7Seeds,2019–2020,"Animation, Action, Adventure",7.108696,"Convinced that Botan has kidnapped them, Natsu...","Stars:Morgan Berry, Amber Lee ...",30.521739,24.695652,NaN,2019,2020
